In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [17]:
year = 2021
cal = 1 # 1 for WT, 2 for UCI, etc.
url = 'https://firstcycling.com/race.php?y='+str(year)+'&t='+str(cal)

r = requests.get(url)
soup = BeautifulSoup(r.text,'html.parser')

tab = soup.find_all('table', class_='sortTabell tablesorter')[1]

refs = [i['href'] for i in tab.find_all('a', href=True)]
races = [x for x in refs if 'race' in x]
urls = ['https://firstcycling.com/'+y for y in races]
results_dict = {}
for race in urls:
    r_race = requests.get(race)
    soup_race = BeautifulSoup(r_race.text,'html.parser')
    race_name = soup_race.find('h1').text
    rows = soup_race.find_all('tbody')[0].find_all('tr')
    result = []
    for tr in rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip()]
        if row:
            result.append(row)
    try:        
        results_dict[race_name] = pd.DataFrame(result, 
                                               columns=['Position', 'Rider', 'Born', 'Team', 'Total time', 'FC points'])
    except ValueError:
        continue

In [18]:
results_dict

{'UAE Tour - 2021':     Position             Rider  Born                   Team Total time  \
 0         01     Tadej Pogacar  1998      UAE Team Emirates   24:00:28   
 1         02        Adam Yates  1992       INEOS Grenadiers       + 35   
 2         03      Joao Almeida  1998  Deceuninck-Quick Step    + 01:02   
 3         04      Chris Harper  1994            Jumbo-Visma    + 01:42   
 4         05   Neilson Powless  1996     EF Education-NIPPO    + 01:45   
 ..       ...               ...   ...                    ...        ...   
 134      DNF     Jos Van Emden  1985            Jumbo-Visma       None   
 135      DNF      Kaden Groves  1998      Team BikeExchange       None   
 136      DNF     Florian Stork  1997               Team DSM       None   
 137      DNF  Matteo Moschetti  1996         Trek-Segafredo       None   
 138      DNF    Antonio Tiberi  2001         Trek-Segafredo       None   
 
     FC points  
 0         300  
 1         250  
 2         215  
 3         